![](1.jpg)

In [3]:
from __future__ import print_function, division
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn  #这是一个可视化的展示库
import random
from sklearn.ensemble import RandomForestRegressor
import sklearn.preprocessing as preprocessing
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
import os

In [4]:
#直接使用pandas对数据进行直观展示观看
data=pd.read_csv('G:\Machine-learning\\tensorflow-train-project(jupyter)\kaggele-project6-tairanprediction\dataset\\train.csv')
print(data.columns)

FileNotFoundError: File b'G:\\Machine-learning\\tensorflow-train-project(jupyter)\\kaggele-project6-tairanprediction\\dataset\\train.csv' does not exist

# 前置函数部分
这里会先准备好补全信息函数(对缺失值的考虑十分重要)，属性值转换函数，数据归一化函数。

In [77]:
#年龄信息补全函数：  这里 使用的方式是RandomForesetClassifier来填补
def set_missing_ages(data):
    age_df=data[['Age','Fare','Parch','SibSp','Pclass']]  #pandas的利用，抽取一部分特征作为年龄相关特征时使用  这里使用五个特征[年龄、船票价格、父母子女数、经济状况]
    #print('age_df:',age_df)
    known_age=age_df[age_df.Age.notnull()].as_matrix()#   这是一种抽离，把知道年龄的数据抽取出来作为一个 矩阵，这个矩阵有包括上面的5个特征值
    #print('known_age:',known_age)
    unknown_age=age_df[age_df.Age.isnull()].as_matrix()  
    
    #接下来使用的事RandomForestClassifier算法来补全年龄特征，这里补全特征的思路是  使用除年龄外的其他特征来作为补全年龄的依据
    y=known_age[:,0]
    x=known_age[:,1:]
    #创建补全算法，并进行数据喂养  随机森林中参数意义为：n_estimators 子模型个数  random_state指定随机器对象 参数可参考https://www.cnblogs.com/amberdata/p/7203632.html  
    rfr=RandomForestRegressor(random_state=0,n_estimators=2000,n_jobs=-1)
    rfr.fit(x,y)
    #使用训练好的随机森林进行预测
    predictedAges=rfr.predict(unknown_age[:,1::])#对不知道年龄的矩阵，使用其特征进行年龄的预测
    data.loc[(data.Age.isnull()),'Age']=predictedAges#这是针对DateFrame的定位并填充残缺值的方式
    return data

In [78]:
#将属性转换为数值的函数
def attribute_to_number(data):
    #前三行会对属性进行独热编码
    dummies_Pclass=pd.get_dummies(data['Pclass'],prefix='Pclass')#get_dummies会对属性进行独热编码
    dummies_Embarked=pd.get_dummies(data['Embarked'],prefix='Embarked')
    dummies_Sex=pd.get_dummies(data['Sex'],prefix='Sex')
    #这里会将独热编码结果放到  组合结果的后面
    data=pd.concat([data,dummies_Pclass,dummies_Embarked,dummies_Sex],axis=1) #concat不管列名,直接加到一起,可以加到后面、也可以加到右边,axis=0为加到后面,axis=1为加到右边
    #针对这个类型，将前面没用的非数值属性去掉(使用后面新加入的 转化好的属性即可)
    data.drop(['Pclass','Sex','Embarked'],axis=1,inplace=True)
    return data
#对属性值进行归一化操作
def Scales(data):
    scaler=preprocessing.StandardScaler()#StandardScaler用于数据标准化，计算训练集的平均值和标准差,以便测试数据集使用相同的变换
    #对年龄进行归一化处理
    age_scale_param=scaler.fit(data['Age'].reshape(-1,1))#这样转换，使其具有0均值，单位方差
    data['Age_scaled']=scaler.fit_transform(data['Age'].reshape(-1,1),age_scale_param)#fit_transform可以进行最大最小的标准化
    #对工资进行归一化处理
    Fare_scale_param=scaler.fit(data['Fare'].reshape(-1,1))
    data['Fare_scaled']=scaler.fit_transform(data['Fare'].reshape(-1,1),Fare_scale_param)
    #对兄弟配偶数进行归一化处理
    SibSp_scale_param=scaler.fit(data['SibSp'].reshape(-1,1))
    data['SibSp_scaled']=scaler.fit_transform(data['SibSp'].reshape(-1,1),SibSp_scale_param)
    #对父母子女数进行归一化处理
    Parch_scale_param=scaler.fit(data['Parch'].reshape(-1,1))
    data['Parch_scaled']=scaler.fit_transform(data['Parch'].reshape(-1,1),SibSp_scale_param)
    #使用归一化后的数据，将之前的数据删去
    data.drop(['Parch','SibSp','Fare','Age'],axis=1,inplace=True)
    return data

# 第一部分
读取数据集之后，现在我们进行数据的预处理，预处理包括提取主要特征（这里经过我们思考，感觉年龄和性别以及座位等级也许是比较中的特征信息，因此我们会将多余的信息删去）、补全数据、数据的归一化这些思路

In [79]:
def DataPreProcess(in_data,submat_flg):#数据预处理    submat_flg为1表示是针对测试数据的处理
    #核心的几个预处理操作，有些函数需要自己手动去写
    in_data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True) #删去无关特征
    data_ages_fitted=set_missing_ages(in_data)#补足年龄信息
    data=attribute_to_number(data_ages_fitted)#类目属性转为数值型
    data_scaled=Scales(data)#将转化为数值型的特征数据进行归一化
    
    #对特征数据和标签数据的划分   进行划分后作为新的数据返回
    data_copy=data_scaled.copy(deep=True)  #将数据集进行完全的备份   深拷贝
    data_copy.drop(['Pclass_1','Pclass_2','Pclass_3','Embarked_C','Embarked_Q','Embarked_S','Sex_female','Sex_male','Age_scaled','Fare_scaled','SibSp_scaled','Parch_scaled'],axis=1,inplace=True)
    data_y=np.array(data_copy)#这里是删去了很多东西(进行一种选择性的删除，还有axis这个自己不太掌握)   剩下的作为y的标签
    if submat_flg==0:
        data_scaled.drop(['Survived'],axis=1,inplace=True)
        data_X=np.array(data_scaled)#特征组合x，这是要进行网络输入所用的所有特征值
        return data_X,data_y
    if(submat_flg==1):
        data_X=np.array(data_scaled)#特征组合x，这是要进行网络输入所用的所有特征值
        return data_X,data_y
        

# 第二部分
基于之前的逻辑回归进行修改，在模型部分更改即可
基本思路：创建变量    用变量去执行网络，主要的区别是之前是使用模型计算得出结果，现在是获取得到网络的softmax输出值。。。   
这里打算创建一个五层的神经网络，三隐层即可  (原型中包装的有问题，全写在一起了，这样不好，因为现在我想拿模型进行预测时发现，擦，我还得再带入模型)

In [80]:
def NN(data_X,data_y):
    training_epochs=3
    learning_rate=0.01
    display_step=2
    batch_size=10
    logs_train_dir="G:\Machine-learning\\tensorflow-train-project(jupyter)\kaggele-project6-tairanprediction\logs\\"
    #同样进行  喂养集和校验集的分离
    X_train,X_test,y_train,y_test=train_test_split(data_X,data_y,test_size=0.2,random_state=0)
    y_train=tf.concat([1-y_train,y_train],1)  #concat用于连接两个矩阵   1是指连接的维度（从哪个角度）
    y_test=tf.concat([1-y_test,y_test],1)
    #创建网络中使用的主要的变量
    n_samples=X_train.shape[0]#得到训练集数量
    n_feature=X_train.shape[1]#进行训练的特征数
    n_class=2#标签的数值代表分类  是2分类
    x=tf.placeholder(tf.float32,[None,n_feature]) #传入float类型的参数即可
    y=tf.placeholder(tf.float32,[None,n_class])
    #做好准备工作， 现在开始正式搭建神经网络的过程
    with tf.name_scope("layer_in"): #第一层的网络设置
        Weight1=tf.Variable(tf.truncated_normal([n_feature,256],stddev=0.1),name='W1')
        bias1=tf.Variable(tf.truncated_normal([256],stddev=0.1),name='b1')
        hidden1=tf.nn.relu(tf.matmul(x,Weight1)+bias1,name='hidden1')
    with tf.name_scope("layer_hidden_1"):
        Weight2=tf.Variable(tf.truncated_normal([256,128],stddev=0.1),name='W2')
        bias2=tf.Variable(tf.truncated_normal([128],stddev=0.1),name='b2')
        hidden2=tf.nn.relu(tf.matmul(hidden1,Weight2)+bias2,name='hidden2')
    with tf.name_scope('layer_hidden_2'):
        Weight3=tf.Variable(tf.truncated_normal([128,64],stddev=0.1),name='W3')
        bias3=tf.Variable(tf.truncated_normal([64],stddev=0.1),name='b3')
        hidden3=tf.nn.relu(tf.matmul(hidden2,Weight3)+bias3,name='hidden3')
    with tf.name_scope('layer_out'):
        Weight4=tf.Variable(tf.truncated_normal([64,2],stddev=0.1),name='W4')
        bias4=tf.Variable(tf.truncated_normal([2],stddev=0.1),name='b4')
        prediction=tf.nn.softmax(tf.matmul(hidden3,Weight4)+bias4,name='y_out')  #二分类获取输出即可
    
    #这里再计算损失值时  为了防止网络太复杂导致过拟合问题，所以我们这里使用正则化方法对求损失值函数进行限制（进行L2正则化）
    vars=tf.trainable_variables()#可以获取所有需要训练的变量列表，这样才能整体对参数进行调整
    lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars if 'b' not in v.name]) * 0.05#设置L2正则化 的式子
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y)+lossL2) 
    
    train_step=tf.train.MomentumOptimizer(learning_rate,0.9).minimize(cost)
    
    correct_prediction=tf.equal(tf.arg_max(prediction,1),tf.arg_max(y,1))#arg_max用于返回vector中 最大值的索引号
    accurary=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    ckpt_dir='./logs'
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    saver=tf.train.Saver()
    init=tf.initialize_all_variables()
    with tf.Session() as sess:       
        tf.global_variables_initializer().run()
        ckpt=tf.train.latest_checkpoint(ckpt_dir)
        if ckpt:
            print('Restoring from checkpoint: %s' % ckpt,'        path:',ckpt.model_checkpoint_path)
            saver.restore(sess,ckpt)

        for epoch in range(training_epochs):
            print('epoch:',epoch)
            avg_cost=0
            total_batch=int(n_samples/batch_size)   #在每个训练批次里面  我们都会使用所有的数据进行训练，所以这里会有个除式计算
            for i in range(total_batch):
                _,c=sess.run([train_step,cost],feed_dict={x:X_train[i*batch_size:(i+1)*batch_size],y:y_train[i*batch_size:(i+1)*batch_size,:].eval()})
                avg_cost=c/total_batch#单批次下的损失值
            plt.plot(epoch+1,avg_cost,'co')#把损失值变化 用plt图标来展示出来
            
            if(epoch+1)%display_step==0:#达到一定批次之后进行准确的输出展示
                print('Epoch:',(epoch+1),',cost=',avg_cost)
                #每个5步数，并进行模型结构和参数的保存
                checkpoint_path=os.path.join(logs_train_dir,'model.ckpt')
                saver.save(sess, ckpt_dir + '/logistic.ckpt')
               # print('the Weight: ',sess.run(W),'  ,the bias:',sess.run(b))
        X_test=sess.run(tf.convert_to_tensor(X_test))
        y_test=sess.run(tf.convert_to_tensor(y_test))
        print('Testing Accuracy:',sess.run(accurary,feed_dict={x:X_test,y:y_test}))#这个地方如果这样用，往feed_dict中传的是张量，这是tf的理解难点
        plt.xlabel('Epoch')
        plt.ylabel('cost')
        plt.show()
    sess.close()
        #W=sess.run(W)
        #b=sess.run(b)        

In [81]:
#调用函数来启动计算过程  
if __name__=="__main__":
    data=pd.read_csv('G:\Machine-learning\\tensorflow-train-project(jupyter)\kaggele-project6-tairanprediction\dataset\\train.csv')#用padas读取csv文件
    submat_test_data=pd.read_csv('G:\Machine-learning\\tensorflow-train-project(jupyter)\kaggele-project6-tairanprediction\dataset\\test.csv')
    submat_test_data_PassengerId=submat_test_data['PassengerId']
    data_x,data_y=DataPreProcess(data,0)
    print('data_x 的格式为:',data_x.shape)
    submat_test_data_x,submat_test_data_y=DataPreProcess(submat_test_data,1)
    print('submat_test_data_x 的格式为:',submat_test_data_x.shape)
    NN(data_x,data_y)
    #print('the W:',W,',it type is',type(W))
    #print('the b:',b,',it type is',type(b))
    

D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .

data_x 的格式为: (891, 12)


D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\ProgramData\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .

submat_test_data_x 的格式为: (418, 12)


AttributeError: 'str' object has no attribute 'model_checkpoint_path'

# 终篇
使用训练出的网络对结果进行预测，读取网络模型，并将网络运行的结果保存在csv文件中去

In [35]:
#自己包装的不好，得再建个函数     输入x，放回最后的预测结果
def test_model(X):
    n_feature=X.shape[1]
    print('n_feature:',n_feature)
    #x=tf.placeholder(tf.float32,[None,n_feature]) #传入float类型的参数即可
    #做好准备工作， 现在开始正式搭建神经网络的过程
    with tf.name_scope("layer_in"): #第一层的网络设置
        print('测试  ！！！！')
        W1=tf.Variable(tf.truncated_normal([12,256],stddev=0.1),name='W1')
        print('测试  ！')
        b1=tf.Variable(tf.truncated_normal([256],stddev=0.1),name='b1')
        print('测试 ')
        hidden1=tf.nn.relu(tf.matmul(X,W1)+b1,name='hidden1')
        print('测试 1')
    with tf.name_scope("layer_hidden_1"):
        print('测试2 ')
        W2=tf.Variable(tf.truncated_normal([256,128],stddev=0.1),name='W2')
        print('测试3 ')
        b2=tf.Variable(tf.truncated_normal([128],stddev=0.1),name='b2')
        print('测试 4')
        hidden2=tf.nn.relu(tf.matmul(hidden1,W2)+b2,name='hidden2')
        print('测试5 ')
    with tf.name_scope('layer_hidden_2'):
        W3=tf.Variable(tf.truncated_normal([128,64],stddev=0.1),name='W3')
        print('测试 6')
        b3=tf.Variable(tf.truncated_normal([64],stddev=0.1),name='b3')
        print('测试 7')
        hidden3=tf.nn.relu(tf.matmul(hidden2,W3)+b3,name='hidden3')
    with tf.name_scope('layer_out'):
        W4=tf.Variable(tf.truncated_normal([64,2],stddev=0.1),name='W4')
        print('测试 9')
        b4=tf.Variable(tf.truncated_normal([2],stddev=0.1),name='b4')
        print('测试 10',b4)
        
        prediction=tf.nn.softmax(tf.matmul(hidden3,W4)+b4,name='y_out')  #二分类获取输出即可
        last_result=tf.arg_max(prediction,1)
        print('测试 11')
    return last_result

#读取保存的网络模型
logs_train_dir="G:\Machine-learning\\tensorflow-train-project(jupyter)\kaggele-project6-tairanprediction\logs\\"
saver=tf.train.Saver()
print('submat_test_data_x  的格式:',submat_test_data_x.shape)
n_features=submat_test_data_x.shape[1]
X=tf.placeholder(tf.float32,[None,n_features])
last_result=test_model(X)
init=tf.initialize_all_variables()
with tf.Session() as sess2:
    sess2.run(init)
    #读取保存的ckpt文件
    
    ckpt = tf.train.latest_checkpoint(logs_train_dir)
    if ckpt:
        print('Restoring from checkpoint: %s' % ckpt)
        saver.restore(sess2, ckpt)

    #ckpt=tf.train.get_checkpoint_state(logs_train_dir)
    #if ckpt and ckpt.model_checkpoint_path:
       # global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
       # saver.restore(sess2,ckpt.model_checkpoint_path)
    #现在已经加载了之前保存的会话了。      
    predictions_to=sess2.run([last_result],feed_dict={X:submat_test_data_x})
    print('predictions:',predictions_to)  
    #print('predictions:',predictions_to.shape)                                                                   
submission=pd.DataFrame({'PassengerId':submat_test_data_PassengerId,'Survived':predictions_to[0]})#构造一个表，用于放置预测结果    predictions_to[0]语句相当于把这取出来了
submission.to_csv('submit.csv',index=False)


#使用计算得到的W和b参数重新启动计算过程     在会话中进行获救结果的预测，并保持在一个csv文件中
# #创建计算图点
# n_features=submat_test_data_x.shape[1]
# X=tf.placeholder(tf.float32,[None,n_features])
# yHat=tf.add(tf.matmul(X,W),B)
# last_result=tf.arg_max(yHat,1)   #构建计算图，直接放置即可    自己想出来的方法
# with tf.Session() as sess2:
#     tf.global_variables_initializer().run()
#     submat_test_data_x=sess2.run(tf.convert_to_tensor(submat_test_data_x))
#     Weight=sess2.run(tf.convert_to_tensor(Weight))
#     bias=sess2.run(tf.convert_to_tensor(bias))
#     print('submat shape:',submat_test_data_x.shape)
#     print('Weight shape:',Weight.shape)
#     print('bias shape:',type(bias))
#     predictions_to=sess2.run([last_result],feed_dict={X:submat_test_data_x})#np.argmax用于得到数组最大值的索引  整体获得预测结果
#     #predictions_to=sess2.run(tf.arg_max(tf.convert_to_tensor(predictions),1))
#     #print('predictions:',predictions_to[0].shape)
# submission=pd.DataFrame({'PassengerId':submat_test_data_PassengerId,'Survived':predictions_to[0]})#构造一个表，用于放置预测结果    predictions_to[0]语句相当于把这取出来了
# submission.to_csv('submit.csv',index=False)

submat_test_data_x  的格式: (418, 12)
n_feature: 12
测试  ！！！！
测试  ！
测试 
测试 1
测试 6
测试 7


ValueError: Dimensions must be equal, but are 256 and 128 for 'layer_hidden_2_9/MatMul' (op: 'MatMul') with input shapes: [?,256], [128,64].

In [74]:
import os
model_dir = 'G:/Machine-learning/tensorflow-train-project(jupyter)/kaggele-project6-tairanprediction/logs/'
from tensorflow.python import pywrap_tensorflow
#checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
checkpoint_path = os.path.join(model_dir, "/logistic")
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("tensor_name: ", key)
    print(reader.get_tensor(key))


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for G:/logistic